In [1]:
# Exp-00
# train classic feedforward networks
# command to run 
# stdbuf -o 0 python Exp-00-Train_classic.py 2>&1 | tee Exp-00-log-2019-09-13-00:56.txt

In [2]:
from __future__ import print_function

In [3]:
import numpy as np
import time
import os
import sys
import GPUtil
import pickle

import torch

sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [4]:
# Preliminaries
# saved in :
save_dir =  "/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/"

In [5]:
# create timestamped folder
timestamp = time.strftime("%Y_%m_%d_at_%H:%M:%S")
print(timestamp)
try:
    os.mkdir(save_dir + '/' + timestamp)
except e:
    pass

2020_01_07_at_15:16:37


In [6]:
num_iter = 5

In [7]:
# load shared configs
shared_config_fixed_parameters_dic = np.load(save_dir+'/shared_config_fixed_parameters_dic.pckl',allow_pickle='TRUE')

In [8]:
shared_config_fixed_parameters_dic = np.load(save_dir+'/shared_config_fixed_parameters_dic.pckl',allow_pickle='TRUE')
print("Shared Config")
print("{}".format(shared_config_fixed_parameters_dic))

Shared Config
{'batch_size': 168, 'num_epochs': 5, 'lmbda_l2': 8e-06, 'opt_lr': 0.001, 'optimizer_name': 'Adam'}


In [9]:
# extract params
batch_size = shared_config_fixed_parameters_dic['batch_size']
batch_size = 50000
optimizer_name = shared_config_fixed_parameters_dic['optimizer_name']
lmbda_l2 = shared_config_fixed_parameters_dic['lmbda_l2']
opt_lr = shared_config_fixed_parameters_dic['opt_lr']
num_epochs = shared_config_fixed_parameters_dic['num_epochs']

In [10]:
# load list of all configs
shared_config_experiments_list = np.load(save_dir+'/shared_config_experiments_list.pckl',allow_pickle='TRUE')

In [11]:
# load data
(mnist_train_loader, mnist_val_loader, mnist_test_loader) = fgnh.mnist_dataloaders(batch_size=batch_size)

In [12]:
#  do 5 iterations
for ite in range(num_iter):
    # for each config
    for config in shared_config_experiments_list:
        
        # extract config values:
        hidden_layer_sizes = config['hidden_layer_sizes']
        drop_p = config['drop_p']
        
        # def model name
        model_name = 'classic_hl{}_dp{}_ite{}'.format(hidden_layer_sizes, drop_p, ite)
        
        print("Current model name: {}".format(model_name))
        
        # create folder
        model_folder = save_dir+'/'+timestamp+'/'+model_name+'/'
        os.mkdir(model_folder)

        # create classic network
        model = fgnl.Feedforward_Classic_net(in_feats=28*28, 
                                             out_feats=10, 
                                             hidden_layer_sizes=hidden_layer_sizes, 
                                             drop_p=drop_p)
        loss  = fgnh.def_classical_cross_ent_loss(lmbda_l2=lmbda_l2)
        
        print("MODEL")
        print(model)
        
        # optimize every params that require grad
        if optimizer_name=='Adam':
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=opt_lr)
        
        # attempt to sent to GPU, else train over CPU
        model_sent_to_device = False
        sleep_time = 30
        while not model_sent_to_device and sleep_time<4800:
            # get free device
            device = torch.device('cuda')
            device_id = GPUtil.getFirstAvailable(order='memory', maxLoad=1.0, maxMemory=0.8, verbose=False)[0]

            # send to least used GPU
            print("Using GPU:", device_id)
            with torch.cuda.device(device_id):
                # send to device
                try:
                    model.to(device)
                    model_sent_to_device=True
                except Exception as e:
                    print(e)
                    sleep_time = 1.66*sleep_time
                    print("Not enough ram. Wait {}s and continue").format(sleep_time)
                    time.sleep(sleep_time)
        
        if not model_sent_to_device:
            print("Failed to send to GPU, using CPU")
            device = torch.device('cpu')
            model.to(device)
            
        
        # save model pre_training
        print("Saving initial (before training) model {} in {}".format(model_name, save_dir))
        # save model entirely
        torch.save(model, model_folder + "/init_classic_model_full.pth")
        # save model weights
        torch.save(model.state_dict(), model_folder + "/init_classic_model_state_dict.pth")
        # save config
        with open(model_folder+"/config.txt", "w") as text_file:
            text_file.write("batch_size {}\n".format(str(batch_size)))
            text_file.write("num_epochs {}\n".format(str(num_epochs)))
            text_file.write("lmbda_l2 {}\n".format(str(lmbda_l2)))
            text_file.write("optimizer_name {}\n".format(optimizer_name))
            text_file.write("opt_lr {}\n".format(opt_lr))
            text_file.write("hidden_layer_sizes {}\n".format(hidden_layer_sizes))
            text_file.write("drop_p {}\n".format(drop_p))

        
        # train model
        training_done = False
        print("Training")
        try:
            train_res = fgnh.train(model=model, train_loader=mnist_train_loader, loss_func=loss, 
                                   optimizer=optimizer, epochs=num_epochs, save_hist=0, 
                                   pred_func=fgnh.cross_ent_pred_accuracy, test_loader=mnist_val_loader, 
                                   verbose=True)
            training_done=True
        except Exception as e:
            print(e)
            print("Training failed")
        
        # training finished, if succesful
        if training_done:
            # save model, histories
            # save trained model
            print("Saving trained model {} in {}".format(model_name, model_folder))

            # save model entirely
            torch.save(model,model_folder+"/trained_classic_model_full.pth")

            # save model weights
            torch.save(model.state_dict(), model_folder+"/trained_classic_model_state_dict.pth")

            # save training histories as pickle
            # WHOLE HISTORIES are too large, only save loss and accuracy
            with open(model_folder+"/train_histories.pckl","wb") as pickle_file:
                pickle.dump(train_res, pickle_file)
        
        # clean up GPU
        del model
        torch.cuda.empty_cache()
        

Current model name: classic_hl[]_dp0.0_ite0
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList()
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=784, out_features=10, bias=True)
)
Using GPU: 0
Saving initial (before training) model classic_hl[]_dp0.0_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/
Training
Epoch 0 Train set - Average loss: 2.3714, Accuracy: 3492/50000 (7%)
Test set - Average loss: 2.2918, Accuracy: 1505/10000 (15%)
Epoch 1 Train set - Average loss: 2.2859, Accuracy: 7492/50000 (15%)
Test set - Average loss: 2.2022, Accuracy: 2270/10000 (23%)
Epoch 2 Train set - Average loss: 2.2029, Accuracy: 12578/50000 (25%)
Test set - Average loss: 2.1195, Accuracy: 3064/10000 (31%)
Epoch 3 Train set - Average loss: 2.1227, Accuracy: 16737/50000 (33%)
Test set - Average loss: 2.0422, Accuracy: 3750/10000 (38%)
Epoch 4 Train set - Average loss: 2.0453, Accuracy: 19815/50

Using GPU: 0
Saving initial (before training) model classic_hl[64]_dp0.4_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/
Training
Epoch 0 Train set - Average loss: 2.3717, Accuracy: 4137/50000 (8%)
Test set - Average loss: 2.2397, Accuracy: 2333/10000 (23%)
Epoch 1 Train set - Average loss: 2.1203, Accuracy: 13695/50000 (27%)
Test set - Average loss: 2.1679, Accuracy: 3693/10000 (37%)
Epoch 2 Train set - Average loss: 1.9052, Accuracy: 24097/50000 (48%)
Test set - Average loss: 2.0949, Accuracy: 4444/10000 (44%)
Epoch 3 Train set - Average loss: 1.7321, Accuracy: 29775/50000 (60%)
Test set - Average loss: 2.0211, Accuracy: 5038/10000 (50%)
Epoch 4 Train set - Average loss: 1.5975, Accuracy: 32710/50000 (65%)
Test set - Average loss: 1.9474, Accuracy: 5491/10000 (55%)
Saving trained model classic_hl[64]_dp0.4_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[64]_dp0.4_ite0/
Current 

Using GPU: 0
Saving initial (before training) model classic_hl[1024]_dp0.4_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/
Training
Epoch 0 Train set - Average loss: 2.3315, Accuracy: 5681/50000 (11%)
Test set - Average loss: 2.0300, Accuracy: 6210/10000 (62%)
Epoch 1 Train set - Average loss: 1.5347, Accuracy: 32580/50000 (65%)
Test set - Average loss: 1.7868, Accuracy: 6822/10000 (68%)
Epoch 2 Train set - Average loss: 1.1018, Accuracy: 36253/50000 (73%)
Test set - Average loss: 1.5763, Accuracy: 7087/10000 (71%)
Epoch 3 Train set - Average loss: 0.8750, Accuracy: 37970/50000 (76%)
Test set - Average loss: 1.3965, Accuracy: 7277/10000 (73%)
Epoch 4 Train set - Average loss: 0.7399, Accuracy: 39318/50000 (79%)
Test set - Average loss: 1.2432, Accuracy: 7454/10000 (75%)
Saving trained model classic_hl[1024]_dp0.4_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[1024]_dp0.4_ite0/
C

Epoch 2 Train set - Average loss: 1.9253, Accuracy: 24337/50000 (49%)
Test set - Average loss: 2.2195, Accuracy: 4184/10000 (42%)
Epoch 3 Train set - Average loss: 1.7918, Accuracy: 29310/50000 (59%)
Test set - Average loss: 2.1853, Accuracy: 5666/10000 (57%)
Epoch 4 Train set - Average loss: 1.6893, Accuracy: 32264/50000 (65%)
Test set - Average loss: 2.1482, Accuracy: 6430/10000 (64%)
Saving trained model classic_hl[64, 64]_dp0.1_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[64, 64]_dp0.1_ite0/
Current model name: classic_hl[64, 64]_dp0.4_ite0
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ib): Ba

Test set - Average loss: 1.3948, Accuracy: 7675/10000 (77%)
Saving trained model classic_hl[1024, 1024]_dp0.0_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[1024, 1024]_dp0.0_ite0/
Current model name: classic_hl[1024, 1024]_dp0.1_ite0
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=1024, out_features=10, bias=True)
)
Using GPU: 0
Saving initial (before training) model classic_hl[1024, 1024]_dp0.1_ite0 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/


...

Using GPU: 0
Saving initial (before training) model classic_hl[16, 16]_dp0.0_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/
Training
Epoch 0 Train set - Average loss: 2.4279, Accuracy: 3690/50000 (7%)
Test set - Average loss: 2.3159, Accuracy: 997/10000 (10%)
Epoch 1 Train set - Average loss: 2.3026, Accuracy: 5925/50000 (12%)
Test set - Average loss: 2.3026, Accuracy: 1074/10000 (11%)
Epoch 2 Train set - Average loss: 2.1928, Accuracy: 9339/50000 (19%)
Test set - Average loss: 2.2875, Accuracy: 1168/10000 (12%)
Epoch 3 Train set - Average loss: 2.1050, Accuracy: 13351/50000 (27%)
Test set - Average loss: 2.2709, Accuracy: 1258/10000 (13%)
Epoch 4 Train set - Average loss: 2.0354, Accuracy: 17128/50000 (34%)
Test set - Average loss: 2.2529, Accuracy: 1344/10000 (13%)
Saving trained model classic_hl[16, 16]_dp0.0_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[16, 16]_dp0.0_ite4/

Epoch 2 Train set - Average loss: 1.9524, Accuracy: 23344/50000 (47%)
Test set - Average loss: 2.2250, Accuracy: 4085/10000 (41%)
Epoch 3 Train set - Average loss: 1.8115, Accuracy: 28955/50000 (58%)
Test set - Average loss: 2.1924, Accuracy: 4788/10000 (48%)
Epoch 4 Train set - Average loss: 1.7018, Accuracy: 32262/50000 (65%)
Test set - Average loss: 2.1571, Accuracy: 5316/10000 (53%)
Saving trained model classic_hl[64, 64]_dp0.4_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[64, 64]_dp0.4_ite4/
Current model name: classic_hl[256, 256]_dp0.0_ite4
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (

Epoch 4 Train set - Average loss: 0.4703, Accuracy: 43074/50000 (86%)
Test set - Average loss: 1.3892, Accuracy: 7872/10000 (79%)
Saving trained model classic_hl[1024, 1024]_dp0.1_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[1024, 1024]_dp0.1_ite4/
Current model name: classic_hl[1024, 1024]_dp0.4_ite4
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=1024, out_features=10, bias=True)
)
Using GPU: 0
Saving initial (before training) model classic_hl[1024, 1024]_dp0.4_ite4 in /home/f

Epoch 2 Train set - Average loss: 1.9473, Accuracy: 26641/50000 (53%)
Test set - Average loss: 2.2739, Accuracy: 3018/10000 (30%)
Epoch 3 Train set - Average loss: 1.8270, Accuracy: 30824/50000 (62%)
Test set - Average loss: 2.2600, Accuracy: 4321/10000 (43%)
Epoch 4 Train set - Average loss: 1.7332, Accuracy: 33428/50000 (67%)
Test set - Average loss: 2.2433, Accuracy: 5410/10000 (54%)
Saving trained model classic_hl[64, 64, 64]_dp0.0_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[64, 64, 64]_dp0.0_ite4/
Current model name: classic_hl[64, 64, 64]_dp0.1_ite4
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

Epoch 0 Train set - Average loss: 2.3324, Accuracy: 5272/50000 (11%)
Test set - Average loss: 2.2790, Accuracy: 3072/10000 (31%)
Epoch 1 Train set - Average loss: 1.7393, Accuracy: 30474/50000 (61%)
Test set - Average loss: 2.2489, Accuracy: 5065/10000 (51%)
Epoch 2 Train set - Average loss: 1.4108, Accuracy: 35531/50000 (71%)
Test set - Average loss: 2.2119, Accuracy: 6110/10000 (61%)
Epoch 3 Train set - Average loss: 1.2024, Accuracy: 37587/50000 (75%)
Test set - Average loss: 2.1667, Accuracy: 6932/10000 (69%)
Epoch 4 Train set - Average loss: 1.0507, Accuracy: 39148/50000 (78%)
Test set - Average loss: 2.1124, Accuracy: 7449/10000 (74%)
Saving trained model classic_hl[256, 256, 256]_dp0.4_ite4 in /home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00//2020_01_07_at_15:16:37/classic_hl[256, 256, 256]_dp0.4_ite4/
Current model name: classic_hl[1024, 1024, 1024]_dp0.0_ite4
MODEL
Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=78

In [13]:
print("SCRIPT END")

SCRIPT END
